In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
from ampligraph.evaluation import train_test_split_no_unseen
from ampligraph.evaluation import evaluate_performance
from ampligraph.latent_features import TransE, ComplEx, ConvE, ConvKB, HolE, DistMult
from ampligraph.utils import save_model, restore_model
import pandas as pd
import numpy as np

In [ ]:
kg = pd.read_csv('./triple/triples05.csv')

In [ ]:
kg = kg.dropna()

In [ ]:
kg['edge'].value_counts()

## For combined emdding

In [ ]:
kg_test = pd.read_csv('user_work_triples.csv')

In [ ]:
kg_test['edge'].value_counts()

In [ ]:
user = list(kg_test['v'].unique())
issue = list(kg_test['u'].unique())

In [ ]:
kg_test = kg_test.dropna()

In [ ]:
_, X_test = train_test_split_no_unseen(np.array(kg_test[kg_test['edge']!='assignee']), test_size=16000, seed=43)

In [ ]:
X_test_df = pd.DataFrame(X_test, columns=['u', 'edge', 'v'])

In [ ]:
index1 = pd.MultiIndex.from_arrays([kg[col] for col in ['u', 'edge', 'v']])
index2 = pd.MultiIndex.from_arrays([X_test_df[col] for col in ['u', 'edge', 'v']])
X_train = kg.loc[~index1.isin(index2)].to_numpy()

In [ ]:
print('Train set size: ', X_train.shape)
print('Test set size: ', X_test.shape)

In [ ]:
model = restore_model(model_name_path = './model/convKB/graph01.model')

In [ ]:
model = ConvKB(batches_count=100, seed=43, epochs=1000, k=200, verbose=True)

In [ ]:
model.fit(X_train)

In [ ]:
save_model(model, model_name_path = './model/convKB/graph01.model')

In [ ]:
filter_triples = np.concatenate((X_train, X_test))

In [ ]:
ranks = evaluate_performance(X_test,
                             model=model, 
                             filter_triples=filter_triples,
                             corrupt_side='o',
                             entities_subset=user,
                             verbose=True)

In [ ]:
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score

mr = mr_score(ranks)
mrr = mrr_score(ranks)

print("MRR: %.2f" % (mrr))
print("MR: %.2f" % (mr))

hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))